# Getting started

Once you've chosen your scenario, download the data from [the Iowa website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [ ]:
# import data and check datatype
import pandas as pd
df = pd.read_csv('~/desktop/liquor10.csv')
df.Date = pd.to_datetime(df.Date)
df.dtypes

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# less than 1% of missing value, not a big deal to drop it
df.dropna(inplace=True)

In [ ]:
# convert dollar columns to numeric datatype
df['cost']=map(lambda x: float(x[1:]), df['State Bottle Cost'])
df['retail']=map(lambda x: float(x[1:]), df['State Bottle Retail'])
df['sale']=map(lambda x: float(x[1:]), df['Sale (Dollars)'])

In [ ]:
# create 'season' feature to use in the model
df['season'] = df.Date.dt.quarter

In [ ]:
# calculate the sum of sale, volume and bottles sold based on location and time
saledata = df.sale.groupby([df['County Number'], df['season']]).sum()
volumedata = df['Volume Sold (Liters)'].groupby([df['County Number'], df['season']]).sum()
bottledata = df['Bottles Sold'].groupby([df['County Number'], df['season']]).sum()
sumdata = pd.concat([saledata, volumedata, bottledata], axis = 1)
sumdata.reset_index(level = ['County Number', 'season'], inplace=True)

In [ ]:
# calculate the average price for different location and season
sumdata['price'] = sumdata.sale/sumdata['Bottles Sold']

In [ ]:
sumdata.head()

# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [ ]:
# plot price vs sale, could identify some outliers

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.jointplot(sumdata.price, sumdata.sale, kind='reg')

In [ ]:
#plot bottles sold vs sale, perfect correlation as expected

sns.jointplot(sumdata['Bottles Sold'], sumdata.sale, kind='reg')

In [ ]:
# drop outliers over 3 standard deviation from the mean
import numpy as np
from scipy import stats
modeldata = sumdata[np.abs(stats.zscore(sumdata)<3).all(axis=1)]

# Refine the data
Look for any statistical relationships, correlations, or other relevant properties of the dataset.

In [ ]:
# check the correlation between features
modeldata.corr()

In [ ]:
# assiagn independent and dependent variables, and create dummies for categorical features

from sklearn import metrics
from sklearn import preprocessing

categorical = preprocessing.OneHotEncoder(categorical_features = [0,1])
X = modeldata[['County Number', 'season', 'price', 'Bottles Sold']]
y = modeldata['sale']
X = categorical.fit_transform(X)

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [ ]:
# build linear regression with lasso regularization to address multicollinearity problem brought by dummies.
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LassoCV, Lasso
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)
lassocv = LassoCV(n_alphas=100, normalize=True, random_state=1)
lassocv.fit(X_train, y_train)
print lassocv.alpha_
y_pred = lassocv.predict(X_test)
print metrics.r2_score(y_test, y_pred)
print np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
plt.scatter(y_pred, y_test)

In [ ]:
# check how the model would perform without bottles sold as independent variable
X = modeldata[['County Number', 'season', 'price']]
y = modeldata['sale']
X = categorical.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

In [ ]:
# r2 is still very high without bottles sold, but the rmse is significantly higher, which indicates:
# 1. the model is less predictive without bottles sold
# 2. r2 may not be a good matrics to evaluate a regularized model since the loss function is not solely mse
lassocv = LassoCV(n_alphas=100, normalize=True, random_state=1)
lassocv.fit(X_train, y_train)
print lassocv.alpha_
pred_y = lassocv.predict(X_test)
print metrics.r2_score(y_test, pred_y)
print np.sqrt(metrics.mean_squared_error(y_test, pred_y))